
# Citation and Conference Data Join

**TODO**

Jupyter Notebook for web scraping of the missing conferences locations of the DBLP dump.

For this process, the following CSV file is needed: ```out_dblp_papers_and_locations.csv```. <br>
The above file must be generated running the ```1 - dblp_add_locations_from_raw_dblp_dump.ipynb``` Notebook that is contained in the same folder as this Noteook.

In particular, the following operations are going to be executed:
* Opening of the CSV peprocessed dump
* Obtaining the missing locations with queries to the DBLP website
* Fix of the locations format

Lastly, the entire preprocessed dump is going to be saved on disk in CSV format

In [1]:
# Libraries Import
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

## File Paths
Please set your working directory paths.

In [2]:
# ******************* PATHS ********************+

# Dumps Directory Path
path_file_import = r'/Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Import/'

# CSV Exports Directory Path
path_file_export = r'/Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Export/'

## Read of the CSV Preprocessed COCI Dump

In [ ]:
df_coci = pd.read_csv(path_file_export + 'out_coci_citations_count.csv', low_memory=False, index_col=[0])
df_coci

Order by citations count descending to see the articles with the most citations

In [ ]:
# Order by citations count descending to see the articles with the most citations
df_coci = df_coci.sort_values(by='citations_count', ascending=False)
df_coci.iloc[:5]

Renaming the article column to doi and making sure that everything is in lowercase:

In [ ]:
df_coci = df_coci.rename(columns={'article': 'doi'})
df_coci = df_coci.reindex(sorted(df_coci.columns), axis=1)

df_coci.doi = df_coci.doi.str.lower()
df_coci.iloc[:5]

## Read of the CSV Preprocessed DBLP Dump

In [3]:
df_dblp = pd.read_csv(path_file_export + 'out_dblp_papers_and_locations_final.csv', low_memory=False, index_col=[0])
df_dblp

,ConferenceLocation,ConferenceTitle,crossref,ee,key,url,year
0,"Eilat, Israel","Cooperative Information Systems, 7th Internati...",conf/coopis/2000,https://doi.org/10.1007/10722620_29,conf/coopis/ChenD00,db/conf/coopis/coopis2000.html,2000
1,"Agia Napa, Cyprus",On the Move to Meaningful Internet Systems 200...,conf/coopis/2004-2,https://doi.org/10.1007/978-3-540-30469-2_46,conf/coopis/AbdellatifCL04,db/conf/coopis/coopis2004-2.html,2004
2,"Eilat, Israel","Cooperative Information Systems, 7th Internati...",conf/coopis/2000,https://doi.org/10.1007/10722620_9,conf/coopis/PapastavrouCSP00,db/conf/coopis/coopis2000.html,2000
3,"Kiawah Island, South Carolina, USA",Proceedings of the Second IFCIS International ...,conf/coopis/97,http://doi.ieeecomputersociety.org/10.1109/COO...,conf/coopis/BultzingsloewenKK97,db/conf/coopis/coopis97.html,1997
4,"Catania, Sicily, Italy",On The Move to Meaningful Internet Systems 200...,conf/coopis/2003,https://doi.org/10.1007/978-3-540-39964-3_50,conf/coopis/GiacolettoA03,db/conf/coopis/coopis2003.html,2003
...,...,...,...,...,...,...,...
2434139,"Thessaloniki, Greece",Philosophy and Theory of Artificial Intelligen...,series/sapere/2013-5,https://doi.org/10.1007/978-3-642-31674-6_9,series/sapere/Besold13,db/series/sapere/sapere5.html,2011
2434140,"Thessaloniki, Greece",Philosophy and Theory of Artificial Intelligen...,series/sapere/2013-5,https://doi.org/10.1007/978-3-642-31674-6_20,series/sapere/Steiner13,db/series/sapere/sapere5.html,2011
2434141,"Thessaloniki, Greece",Philosophy and Theory of Artificial Intelligen...,series/sapere/2013-5,https://doi.org/10.1007/978-3-642-31674-6_25,series/sapere/Armstrong13,db/series/sapere/sapere5.html,2011
2434142,"Thessaloniki, Greece",Philosophy and Theory of Artificial Intelligen...,series/sapere/2013-5,https://doi.org/10.1007/978-3-642-31674-6_12,series/sapere/Freed13,db/series/sapere/sapere5.html,2011


### Removing the DOI Website Path from the DOI URL (ee Column)

First we need to fix the rows that have multiple URLs separated by the | character

In [4]:
for dblp_index, dblp_row in df_dblp.iterrows():

    if str(dblp_row['ee']).split('|').__len__() >= 2:
        df_dblp.at[dblp_index, 'ee'] = dblp_row['ee'].split('|')[1]

Removing the DOI website path

In [5]:
df_dblp['doi'] = np.nan
df_dblp.doi = df_dblp.ee.str.lower().str.split('.org/').str[1]

# Column sort
df_dblp = df_dblp.reindex(sorted(df_dblp.columns), axis=1)

df_dblp.loc[:5]

,ConferenceLocation,ConferenceTitle,crossref,doi,ee,key,url,year
0,"Eilat, Israel","Cooperative Information Systems, 7th Internati...",conf/coopis/2000,10.1007/10722620_29,https://doi.org/10.1007/10722620_29,conf/coopis/ChenD00,db/conf/coopis/coopis2000.html,2000
1,"Agia Napa, Cyprus",On the Move to Meaningful Internet Systems 200...,conf/coopis/2004-2,10.1007/978-3-540-30469-2_46,https://doi.org/10.1007/978-3-540-30469-2_46,conf/coopis/AbdellatifCL04,db/conf/coopis/coopis2004-2.html,2004
2,"Eilat, Israel","Cooperative Information Systems, 7th Internati...",conf/coopis/2000,10.1007/10722620_9,https://doi.org/10.1007/10722620_9,conf/coopis/PapastavrouCSP00,db/conf/coopis/coopis2000.html,2000
3,"Kiawah Island, South Carolina, USA",Proceedings of the Second IFCIS International ...,conf/coopis/97,10.1109/coopis.1997.613823,https://doi.org/10.1109/COOPIS.1997.613823,conf/coopis/BultzingsloewenKK97,db/conf/coopis/coopis97.html,1997
4,"Catania, Sicily, Italy",On The Move to Meaningful Internet Systems 200...,conf/coopis/2003,10.1007/978-3-540-39964-3_50,https://doi.org/10.1007/978-3-540-39964-3_50,conf/coopis/GiacolettoA03,db/conf/coopis/coopis2003.html,2003
5,"Agia Napa, Cyprus",On the Move to Meaningful Internet Systems 200...,conf/coopis/2004-1,10.1007/978-3-540-30468-5_35,https://doi.org/10.1007/978-3-540-30468-5_35,conf/coopis/BuccafurriL04,db/conf/coopis/coopis2004-1.html,2004


### Creation of the DBLP Conference Instance Identifier
Before we can proceed with the join operations, we first need to create a conference identifier that will allow us to join the conference data between DBLP and MAG.

The chosen identifier is going to be composed in the same way of the ConferenceNormalizedName field of the MAG dump: name of the conference series (lower case) + year.

#### Extraction of the Conference Series Name from the Column "key"

In [6]:
df_dblp['ConferenceNormalizedName'] = np.nan
df_dblp.ConferenceNormalizedName = df_dblp.key.str.split('/').str[1]

# Column sort
df_dblp = df_dblp.reindex(sorted(df_dblp.columns), axis=1)
df_dblp.loc[:5]

,ConferenceLocation,ConferenceNormalizedName,ConferenceTitle,crossref,doi,ee,key,url,year
0,"Eilat, Israel",coopis,"Cooperative Information Systems, 7th Internati...",conf/coopis/2000,10.1007/10722620_29,https://doi.org/10.1007/10722620_29,conf/coopis/ChenD00,db/conf/coopis/coopis2000.html,2000
1,"Agia Napa, Cyprus",coopis,On the Move to Meaningful Internet Systems 200...,conf/coopis/2004-2,10.1007/978-3-540-30469-2_46,https://doi.org/10.1007/978-3-540-30469-2_46,conf/coopis/AbdellatifCL04,db/conf/coopis/coopis2004-2.html,2004
2,"Eilat, Israel",coopis,"Cooperative Information Systems, 7th Internati...",conf/coopis/2000,10.1007/10722620_9,https://doi.org/10.1007/10722620_9,conf/coopis/PapastavrouCSP00,db/conf/coopis/coopis2000.html,2000
3,"Kiawah Island, South Carolina, USA",coopis,Proceedings of the Second IFCIS International ...,conf/coopis/97,10.1109/coopis.1997.613823,https://doi.org/10.1109/COOPIS.1997.613823,conf/coopis/BultzingsloewenKK97,db/conf/coopis/coopis97.html,1997
4,"Catania, Sicily, Italy",coopis,On The Move to Meaningful Internet Systems 200...,conf/coopis/2003,10.1007/978-3-540-39964-3_50,https://doi.org/10.1007/978-3-540-39964-3_50,conf/coopis/GiacolettoA03,db/conf/coopis/coopis2003.html,2003
5,"Agia Napa, Cyprus",coopis,On the Move to Meaningful Internet Systems 200...,conf/coopis/2004-1,10.1007/978-3-540-30468-5_35,https://doi.org/10.1007/978-3-540-30468-5_35,conf/coopis/BuccafurriL04,db/conf/coopis/coopis2004-1.html,2004


#### Fix of the Wrong Conferences Series Names
The above operation works for the most of the rows. However, there are some special cases that we need to fix manually.

Fix of the papers that need the extraction from the crossref field:

In [7]:
crossref_list = ['agp', 'aipl', 'amcc', 'amec', 'bcd', 'biotec', 'calculemus', 'cf', 'cnps', 'corr', 'cw', 'daglib', 'dali', 'dbtel', 'ecoopwException', 'express', 'f-egc', 'icsengt', 'ifip5-5', 'infinity', 'isdt', 'iwlcs', 'npiv', 'wcit', 'isdt', 'dsai', 'aipl', 'dbtel', 'cw']

Special cases:

In [15]:
print_counter = 0

for dblp_index, dblp_row in df_dblp.iterrows():

    print_counter += 1
    if print_counter == 25000:
        print(f"Riga numero {dblp_index + 1}")
        print_counter = 0

    fixed = False

    for i in range (0, crossref_list.__len__()):
        if str(dblp_row['crossref']).split('/').__len__() >= 2:
            if crossref_list[i] == str(dblp_row['crossref']).split('/')[1]:
                df_dblp.at[dblp_index, 'ConferenceNormalizedName'] = crossref_list[i]
                fixed = True
                break
    
    if not fixed:
        if "ecoopwException" in str(dblp_row['crossref']):
            df_dblp.at[dblp_index, 'ConferenceNormalizedName'] = "ecoopw"
        elif "icsengt" in str(dblp_row['crossref']):
            df_dblp.at[dblp_index, 'ConferenceNormalizedName'] = "icset"

        if "ali2" in str(dblp_row['key']):
            df_dblp.at[dblp_index, 'ConferenceNormalizedName'] = "ali"
        elif "ictcs2" in str(dblp_row['key']):
            df_dblp.at[dblp_index, 'ConferenceNormalizedName'] = "ictcs" 


Riga numero 25000
Riga numero 50000
Riga numero 75000
Riga numero 100000
Riga numero 125000
Riga numero 150000
Riga numero 175000
Riga numero 200000
Riga numero 225000
Riga numero 250000
Riga numero 275000
Riga numero 300000
Riga numero 325000
Riga numero 350000
Riga numero 375000
Riga numero 400000
Riga numero 425000
Riga numero 450000
Riga numero 475000
Riga numero 500000
Riga numero 525000
Riga numero 550000
Riga numero 575000
Riga numero 600000
Riga numero 625000
Riga numero 650000
Riga numero 675000
Riga numero 700000
Riga numero 725000
Riga numero 750000
Riga numero 775000
Riga numero 800000
Riga numero 825000
Riga numero 850000
Riga numero 875000
Riga numero 900000
Riga numero 925000
Riga numero 950000
Riga numero 975000
Riga numero 1000000
Riga numero 1025000
Riga numero 1050000
Riga numero 1075000
Riga numero 1100000
Riga numero 1125000
Riga numero 1150000
Riga numero 1175000
Riga numero 1200000
Riga numero 1225000
Riga numero 1250000
Riga numero 1275000
Riga numero 1300000
Ri

#### Adding the Year to the Conference Series Name

In [16]:
df_dblp.ConferenceNormalizedName = df_dblp.ConferenceNormalizedName + ' ' + df_dblp.year.astype(str)

In [17]:
df_dblp.loc[:5]

,ConferenceLocation,ConferenceNormalizedName,ConferenceTitle,crossref,doi,ee,key,url,year
0,"Eilat, Israel",coopis 2000,"Cooperative Information Systems, 7th Internati...",conf/coopis/2000,10.1007/10722620_29,https://doi.org/10.1007/10722620_29,conf/coopis/ChenD00,db/conf/coopis/coopis2000.html,2000
1,"Agia Napa, Cyprus",coopis 2004,On the Move to Meaningful Internet Systems 200...,conf/coopis/2004-2,10.1007/978-3-540-30469-2_46,https://doi.org/10.1007/978-3-540-30469-2_46,conf/coopis/AbdellatifCL04,db/conf/coopis/coopis2004-2.html,2004
2,"Eilat, Israel",coopis 2000,"Cooperative Information Systems, 7th Internati...",conf/coopis/2000,10.1007/10722620_9,https://doi.org/10.1007/10722620_9,conf/coopis/PapastavrouCSP00,db/conf/coopis/coopis2000.html,2000
3,"Kiawah Island, South Carolina, USA",coopis 1997,Proceedings of the Second IFCIS International ...,conf/coopis/97,10.1109/coopis.1997.613823,https://doi.org/10.1109/COOPIS.1997.613823,conf/coopis/BultzingsloewenKK97,db/conf/coopis/coopis97.html,1997
4,"Catania, Sicily, Italy",coopis 2003,On The Move to Meaningful Internet Systems 200...,conf/coopis/2003,10.1007/978-3-540-39964-3_50,https://doi.org/10.1007/978-3-540-39964-3_50,conf/coopis/GiacolettoA03,db/conf/coopis/coopis2003.html,2003
5,"Agia Napa, Cyprus",coopis 2004,On the Move to Meaningful Internet Systems 200...,conf/coopis/2004-1,10.1007/978-3-540-30468-5_35,https://doi.org/10.1007/978-3-540-30468-5_35,conf/coopis/BuccafurriL04,db/conf/coopis/coopis2004-1.html,2004


### Write of the Join Ready DBLP Dump on Disk

In [18]:
# Write of the resulting CSV on Disk
df_dblp.to_csv(path_file_export + 'out_dblp_join_ready.csv')
print(f'Successfully Exported the Preprocessed CSV to {path_file_export}out_dblp_join_ready.csv')

Successfully Exported the Preprocessed CSV to /Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Export/out_dblp_join_ready.csv


## Read of the CSV Preprocessed MAG Dump

In [19]:
df_mag = pd.read_csv(path_file_export + 'out_mag_citations_and_locations_final.csv', low_memory=False, index_col=[0])
df_mag.drop(df_mag.filter(regex="Unname"), axis=1, inplace=True)
df_mag

,CitationCount,ConferenceLocation,ConferenceNormalizedName,ConferenceSeriesDisplayName,ConferenceSeriesNormalizedName,Doi,EstimatedCitation,OriginalTitle,PaperTitle,Year
0,12,"Austin, TX",disc 2014,International Symposium on Distributed Computing,DISC,10.1007/978-3-662-45174-8_28,12,The Adaptive Priority Queue with Elimination a...,the adaptive priority queue with elimination a...,2014
1,10,"Wrocław, Poland",esa 2014,European Symposium on Algorithms,ESA,10.1007/978-3-662-44777-2_60,10,Document Retrieval on Repetitive Collections,document retrieval on repetitive collections,2014
2,20,"Innsbruck, Austria",enter 2013,Information and Communication Technologies in ...,ENTER,10.1007/978-3-319-03973-2_13,20,SoCoMo Marketing for Travel and Tourism,socomo marketing for travel and tourism,2013
3,0,"Provence, France",dexa 2002,Database and Expert Systems Applications,DEXA,10.1007/3-540-46146-9_77,0,Similarity Image Retrieval System Using Hierar...,similarity image retrieval system using hierar...,2002
4,19,"Zakopane, Poland",icaisc 2006,International Conference on Artificial Intelli...,ICAISC,10.1007/11785231_94,19,Leukemia prediction from gene expression data—...,leukemia prediction from gene expression data ...,2006
...,...,...,...,...,...,...,...,...,...,...
4409807,0,Singapore,iecon 2020,Conference of the Industrial Electronics Society,IECON,10.1109/IECON43393.2020.9254316,0,Loss Reduction by Synchronous Rectification in...,loss reduction by synchronous rectification in...,2020
4409808,0,"Paris, France",bmsb 2020,International Symposium on Broadband Multimedi...,BMSB,10.1109/BMSB49480.2020.9379806,0,Data Over Cable Services – Improving the BICM ...,data over cable services improving the bicm ca...,2020
4409809,0,NaN,acc 1988,American Control Conference,ACC,10.1109/ACC.1988.4172843,0,Model Reference Robust Adaptive Control withou...,model reference robust adaptive control withou...,1988
4409810,0,"Orlando, Florida, USA",icassp 2002,"International Conference on Acoustics, Speech,...",ICASSP,10.1109/ICASSP.2002.1005676,0,Missing data speech recognition in reverberant...,missing data speech recognition in reverberant...,2002


Order by citations count descending to see the articles with the most citations

In [ ]:
# Order by citations count descending to see the articles with the most citations
df_mag = df_mag.sort_values(by='CitationCount', ascending=False)
df_mag.iloc[:5]

## Join of the New Location Data with the Original Dataframe

In [ ]:
# Merge with the location dataframe
df_dblp_preprocessed = pd.merge(df_dblp_preprocessed, df_conf_locations, on=['url'], how='left')

# Combine the two columns
df_dblp_preprocessed['ConferenceLocation_x'] = df_dblp_preprocessed['ConferenceLocation_x'].fillna(df_dblp_preprocessed['ConferenceLocation_y'])
df_dblp_preprocessed.rename(columns = {'ConferenceLocation_x':'ConferenceLocation'}, inplace=True)
df_dblp_preprocessed = df_dblp_preprocessed.drop(columns=['ConferenceLocation_y'])

df_dblp_preprocessed.iloc[:5]

Count of how many paper's conference locations are still missing

In [ ]:
n_missing = len(df_dblp_preprocessed.index) - len(df_dblp_preprocessed.dropna(subset = ['ConferenceLocation']).index)
print(f"{n_missing} missing paper's conference locations")

## Cleaning the Conference Locations
Some conferences are divided in separated volumes. The volume number is usually indicated in the header of the DBLP page, following the conference location.

For example: *OTM 2004: Agia Napa, Cyprus - Volume 2*

We need to filter it.

In [ ]:
df_dblp_preprocessed.ConferenceLocation = df_dblp_preprocessed.ConferenceLocation.str.split(' - ').str[0]

df_dblp_preprocessed.iloc[:5]

## Write of the Final CSV on Disk

Saving the resulting dataframe on disk in CSV format.

In [ ]:
# Write of the resulting CSV on Disk
df_dblp_preprocessed.to_csv(path_file_export + 'out_dblp_papers_and_locations_final.csv')
print(f'Successfully Exported the Preprocessed CSV to {path_file_export}out_dblp_papers_and_locations_final.csv')

Check of the Exported CSV to be sure that everything went fine.

In [ ]:
# Check of the Exported CSV
df_dblp_exported_csv = pd.read_csv(path_file_export + 'out_dblp_papers_and_locations_final.csv', low_memory=False, index_col=[0])
df_dblp_exported_csv